# Speeding things up: Neighborlists and Dangerous Builds


### Cutoffs
Recall that we typically apply a cutoff to pair potentials once the numerical value of the pair potential becomes very small. E.g., for the Lennard-Jones potential:


$U(r) = 4\epsilon\left[ \left(\frac{\sigma}{r}\right)^{12}- \left(\frac{\sigma}{r}\right)^6 \right ], \quad where \quad r < r_c$ 

$U(r) = 0, \quad where \quad r  \ge r_c$

Where $r_c$ is often $2.5\sigma$, as in the example simulation we just demonstrated with HOOMD-Blue. 

By cutting off the interaction, we avoid having to calculate the energy/force between pair of atoms beyond the cutoff. 

As an example, the python code below compare the number of interactions that would be calculated with and without a cutoff for 1000 particles randomly added to a 10x10x10 box. 

In [ ]:
import numpy
import random
import math 

n_particles = 1000
cutoff = 2.5
cutoff2 = cutoff*cutoff

xyz = numpy.zeros((n_particles,3))
L = numpy.array([10.0,10.0,10.0])
invL = numpy.array([1.0/L[0],1.0/L[1],1.0/L[2]])


def init_particles():
    #set a seed so every run uses the same particle array for consistency
    random.seed(12345)
    
    for i in range(0,n_particles):
        #generate coordinates in a box of dimension L[0]*L[1]*L[2]
        #note particles are free to overlap, this is simply for comparison
        xyz[i][0] = L[0]*random.random()
        xyz[i][1] = L[1]*random.random()
        xyz[i][2] = L[2]*random.random()

#very naive loop over particles
def simple_loop():
    sum_energy = 0
    counter_total = 0
    counter_cutoff = 0

    for i in range (0, n_particles):
        for j in range(i, n_particles):
            dx = xyz[i][0]-xyz[j][0]
            dy = xyz[i][1]-xyz[j][1]
            dz = xyz[i][2]-xyz[j][2]

            #apply pbc
            dx = dx-L[0]*round(dx*invL[0])
            dy = dy-L[1]*round(dy*invL[1])
            dz = dz-L[2]*round(dz*invL[2])
            
            r2 = dx*dx+dy*dy+dz*dz
            counter_total = counter_total+1
            if r2 < cutoff2:
                counter_cutoff = counter_cutoff +1
                
    print("number of energy calculations with no cutoff: ", counter_total)
    print("number of energy calculations with with cutoff: ", counter_cutoff)
    print("reduction factor: ", counter_total/counter_cutoff)
      
init_particles()
simple_loop()

In the above example, we were able to reduce the number of energy calculations needed by a factor of >14. 
However, even though we avoid calculating the energy between these particles, we still have to determine the separation between them to know if they are beyond the cutoff.  This distance calculation is itself quite costly (since we calculate r from the cartesian coordinates).  Applying this calculation to an entire system scales as $O(N^2)$, where $N$ is the number of particles.

### Simple, brute force neighborlist
Recall that, to avoid having to perform this distance calculation every timesteps, a neighborlist can instead be constructed that includes all particles within the cutoff + buffer (the buffer is often called the "skin"). The general idea is that the local environment of a particle changes rather slowly, and thus the same neighborlist can be applied multiple times before needing to be reconstructed, thus reducing the computation. The construction of the neighborlist is still $O(N^2)$ (i.e., it's often called "brute force"), but it is done less frequently thus providing speed improvement. 


### Cell list-based neighborlist
To improve upon the brute force neighborlist, a cell list can first be constructed, and this cell list used to generate the neighborlist.  A cell list is constructed by gridding the system up into smaller boxes, where each edge length is typically $\ge r_c+buffer$.  Cell lists can be constructed relatively inexpensively by binning particles in each direction. The neighborlist is then constructed for a given particle by looping over the cell it is in and it's neighbor cells (note, since pair potentials are pair-wise additive, we only need to consider half of the cells; that is, if A is a neighbor of B, then B is a neighbor of A). 

For efficiency, some codes allow multiple cell lists in a system; this can be especially useful if your system has interactions/particle sizes that are very different; if a single neighborlist were used, it would need to be based on the largest interaction cutoff, thus potentially negating speed increases for particles with shorter range interactions. 

### Optimizing 

The frequency of reconstruction (and hence the speed up) will depend on numerous factors:

How fast  the system is changing. 
> This is often a consequence of the phase, where, for example gas phases will change much more rapidly than dense fluids.  Similarly, temperature will play a role, where high temperature systems will have particles moving faster than low temperature. 

The size of the buffer/skin. 
> If this is set too small, the neighborlist will need to be rebuilt very frequently, if too large, the computational savings will be minimal as the list will contain a very large number of particles.   

The neighborlist has two basic parameters: (1) the skin size (i.e, buffer) and (2) the frequency of updating. These two parameters are coupled.

#### The Skin (Buffer)
The skin is used to create a buffer of particles that are close by, but just outside of the interaction cutoff. This increases the number of calculations that must be done, since we still need to calculate the distance with all particles in the neighborlist even if a particle is ultimately be outside of the interaction range. However, by including this buffer of particles, neighborlists need to be updated less frequently, since we are basically keeping track of the location of nearby particles that we may interact with in the near future.


#### Updating Frequency

In most codes, you have two ways of enforcing an update of the neighborlist.

For example, one approach is to specify that the neighborlist be updated every N steps. This approach tends to work well for systems that are dense or slowly moving, as particle motion should be relatively uniform in the system. However, this may make it challenging to balance wanting to reduce the number of updates that are done and ensuring that particles don't move too far, resulting in missed interactions (i.e., dangerous builds, discussed below). This approach can speed things up, but may be "less safe".

Another approach is to check the displacement of particles. The general rule is that a neighborlist needs to be rebuilt if any particle has move more than skin/2.0, such to ensure that particle interactions are not being missed. This helps to ensure that "dangerous builds" aren't present in the system (see below). In many codes this also can be couple "checking" with a time modulation, e.g., only check every N timesteps or check every N steps and rebuild every M.

#### Dangerous Builds

Most codes will output a summary of the total number of dangerous builds, that is, the total number of times the neighborlist was rebuilt with particles move greater than skin/2.0. In such cases, interactions may have been missed, possibly leading to incorrect behavior.  

For example, the output from a simulation code may look something similar to:

> -- Neighborlist stats:
6392 normal updates / 167 forced updates / 0 dangerous updates <br>
n_neigh_min: 16 / n_neigh_max: 100 / n_neigh_avg: 64.63 <br>
shortest rebuild period: 6<br>

where it is clear that we did not have any dangerous builds. This also provides information regarding the minimum time between neighborlist updates. In more extreme cases of missed interactions, systems may crash due to particles moving outside the bounds of the simulation cell, usually due to extremely high forces on the particles due to overlap. 

Often more detailed information can be acquired.  For example, in the output simulation below, 50% of the computational time is actually spent on the neighborlist.

> `Simulation:     3.1s | 100.0%  `<br>
  `      Integrate:     0.5s | 16.5% `<br>
  `              NVT step 1:     0.1s | 4.2% `<br>
  `              NVT step 2:     0.1s | 3.0% `<br>
  `              Net force:      0.1s | 2.9% `<br>
  `              Thermo:         0.1s | 3.7% `<br>
  `              Self:           0.1s | 2.8% `<br> 
  `      Neighbor:      1.6s | 50.0% `<br>
  `              Cell:           0.0s | 0.8% `<br>
  `                      compute:     0.0s | 0.5% `<br>
  `                      init:        0.0s | 0.2% `<br>
  `              compute:        1.5s | 48.2% `<br>
  `              dist-check:     0.0s | 1.0% `<br> 
  `      Pair lj:       0.9s | 30.2% `<br>
  `      SFCPack:       0.1s | 3.1% `<br>
  `      Self:          0.0s | 0.2% `<br>

### Exercises

1) Using the simple monoatomic LJ system (shown below and similar to the Anatomy of a Script file), adjust the nlist parameters.
- change the buffer (r_buff) to examine the performance as a function of increasing/decreasing the skin.  How small can you make the skin before dangerous builds are detected? How does this impact the speed of the code (e.g. TPS).

 See: https://hoomd-blue.readthedocs.io/en/latest/module-md-nlist.html#hoomd.md.nlist.Cell
 
2) What happens if you set the check period to 1? 10? 100? 1000? Can you speed up the simulation and still avoid dangerous builds?

3) What does the neightborlist auto-tuning function give you (note, you must define this after the integrate call)?

4) How does temperature (kT) influence the parameters you need and number of rebuilds?

In [ ]:
import hoomd
import gsd.hoomd


cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

#########################
# system initialization
#########################

sim.create_state_from_gsd(filename='datafiles/lj.gsd')

#########################
# interaction definition
#########################

cell = hoomd.md.nlist.Cell(buffer=1.0, rebuild_check_delay=1)
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('LJ', 'LJ')] = dict(epsilon=1, sigma=1)
lj.r_cut[('LJ', 'LJ')] = 2.5

#########################
# integrator setup
#########################

nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator = hoomd.md.Integrator(dt=0.005)
integrator.forces.append(lj)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

#########################
# runtime parameters
#########################
logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities=['timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=1000),
                          logger=logger)
sim.operations.writers.append(table)
sim.run(10000)


print("simulation complete")
#delete the instances we defined ensure writers are closed
del sim, table, logger
del integrator, nvt, lj, cell, cpu